In [2]:
import os

In [47]:
articles=os.listdir('articles')
maxlen=35

In [32]:
def doc_stream():
    for file in articles:
        if file[-3:]=='txt':
            with open('articles/'+file, 'r') as f: 
                content=f.read()
                step=4
                sentences=[]
                next_char=[]
                
                for i in range(0, len(content)-maxlen, step):
                    sentences.append(content[i:i+maxlen])
                    next_char.append(content[maxlen+i])
                    
                yield sentences, next_char, content

In [33]:
X=[]
y=[]
contents=''
for sentences, nexts, content in doc_stream():
    X= X + sentences
    y+=nexts
    contents+=content

In [29]:
import numpy as np

In [34]:
chars=sorted(list(set(contents)))
char_index = dict((c,i) for i, c in enumerate(chars))
index_char = dict((i, c) for i, c in enumerate(chars))

In [35]:
len(chars)

121

In [39]:
X_matrix = np.zeros((len(X), maxlen, len(chars)), dtype=np.bool)
y_matrix = np.zeros((len(X), len(chars)), dtype=np.bool)

In [40]:
for i, sentence in enumerate(X):
    for t, char in enumerate(sentence):
        X_matrix[i, t, char_index[char]]=1
    y_matrix[i, char_index[y[i]]]=1

In [42]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import LambdaCallback

In [45]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(256, activation='sigmoid'))
model.add(Dense(len(chars), activation='softmax'))
optimizer=RMSprop()

model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [46]:
def sample(preds, temperature=1.0):
    preds=np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
def on_epoch_end(epoch, _):
    print()
    print('-------- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(contents) - maxlen -1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('------- Diversity:', diversity)
        
        generated = ''
        sentence = contents[start_index: start_index+ maxlen]
        generated += sentence
        print('--------- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)
        
        for i in range(400):
            